<a href="https://colab.research.google.com/github/NolanRink/CS4540/blob/main/HW13/WaveNet_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is notebook gives a quick overview of this WaveNet implementation, i.e. creating the model and the data set, training the model and generating samples from it.

In [1]:
!git clone https://github.com/Vichoko/pytorch-wavenet.git

fatal: destination path 'pytorch-wavenet' already exists and is not an empty directory.


In [2]:
import sys
sys.path.append("pytorch-wavenet")

import torch
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *

## Model
This is an implementation of WaveNet as it was described in the original paper (https://arxiv.org/abs/1609.03499). Each layer looks like this:

```
            |----------------------------------------|      *residual*
            |                                        |
            |    |-- conv -- tanh --|                |
 -> dilate -|----|                  * ----|-- 1x1 -- + -->  *input*
                 |-- conv -- sigm --|     |
                                         1x1
                                          |
 ---------------------------------------> + ------------->  *skip*
```

Each layer dilates the input by a factor of two. After each block the dilation is reset and start from one. You can define the number of layers in each block (``layers``) and the number of blocks (``blocks``). The blocks are followed by two 1x1 convolutions and a softmax output function.
Because of the dilation operation, the independent output for multiple successive samples can be calculated efficiently. With ``output_length``, you can define the number these outputs. Empirically, it seems that a large number of skip channels is required.

In [3]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()
if use_cuda:

    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

use gpu


In [4]:
model = WaveNetModel(layers=8,
                     blocks=2,
                     dilation_channels=24,
                     residual_channels=24,
                     skip_channels=64,
                     end_channels=32,
                     output_length=16,
                     dtype=dtype,
                     bias=True)
# model = load_latest_model_from('snapshots', use_cuda=use_cuda)

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

model:  WaveNetModel(
  (filter_convs): ModuleList(
    (0-15): 16 x Conv1d(24, 24, kernel_size=(2,), stride=(1,))
  )
  (gate_convs): ModuleList(
    (0-15): 16 x Conv1d(24, 24, kernel_size=(2,), stride=(1,))
  )
  (residual_convs): ModuleList(
    (0-15): 16 x Conv1d(24, 24, kernel_size=(1,), stride=(1,))
  )
  (skip_convs): ModuleList(
    (0-15): 16 x Conv1d(24, 64, kernel_size=(1,), stride=(1,))
  )
  (start_conv): Conv1d(256, 24, kernel_size=(1,), stride=(1,))
  (end_conv_1): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
  (end_conv_2): Conv1d(32, 256, kernel_size=(1,), stride=(1,))
)
receptive field:  511
parameter count:  89528


/content/pytorch-wavenet/wavenet_modules.py:53: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:78.)
  self.data = Variable(dtype(num_channels, max_length).zero_())


## Data Set
To create the data set, you have to specify a path to a data set file. If this file already exists it will be used, if not it will be generated. If you want to generate the data set file (a ``.npz`` file), you have to specify the directory (``file_location``) in which all the audio files you want to use are located. The attribute ``target_length`` specifies the number of successive samples are used as a target and corresponds to the output length of the model. The ``item_length`` defines the number of samples in each item of the dataset and should always be ``model.receptive_field + model.output_length - 1``.

```
          |----receptive_field----|
                                |--output_length--|
example:  | | | | | | | | | | | | | | | | | | | | |
target:                           | | | | | | | | | |  
```
To create a test set, you should define a ``test_stride``. Then each ``test_stride``th item will be assigned to the test set.

In [5]:
data = WavenetDataset(dataset_file='pytorch-wavenet/train_samples/bach_chaconne/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='pytorch-wavenet/train_samples/bach_chaconne',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')

one hot input
the dataset has 598436 items


## Training and Logging
This implementation supports logging with TensorBoard (you need to have TensorFlow installed). You can even generate audio samples from the current snapshot of the model during training. This will happen in a background thread on the cpu, so it will not interfere with the actual training but will be rather slow. If you don't have TensorFlow, you can use the standard logger that will print out to the console.
The trainer uses Adam as default optimizer.

In [6]:
def generate_and_log_samples(step):
    sample_length=32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False)
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")


# logger = TensorboardLogger(log_interval=200,
#                            validation_interval=400,
#                            generate_interval=1000,
#                            generate_function=generate_and_log_samples,
#                            log_dir="logs/chaconne_model")

logger = Logger(log_interval=200,
                validation_interval=400,
                generate_interval=1000)

In [10]:
!mkdir -p /content/pytorch-wavenet/train_samples/bach_chaconne

!wget -q \
    https://raw.githubusercontent.com/vincentherrmann/pytorch-wavenet/master/train_samples/bach_chaconne/dataset.npz \
    -O /content/pytorch-wavenet/train_samples/bach_chaconne/dataset.npz

In [ ]:
model.cuda()
trainer = WavenetTrainer(model=model,
                         dataset=data,
                         lr=0.001,
                         snapshot_path='/content/pytorch-wavenet/snapshots',
                         snapshot_name='chaconne_model',
                         snapshot_interval=1000,
                         logger=logger,
                         dtype=dtype,
                         ltype=ltype)

import os
print("CWD is now", os.getcwd())
print('start training...')
trainer.train(batch_size=16,epochs=2)

CWD is now /content
start training...
epoch 0
one training step does take approximately 0.21059401750564577 seconds)
loss at step 200: tensor(6.7628, device='cuda:0', grad_fn=<DivBackward0>)
loss at step 400: tensor(3.3880, device='cuda:0', grad_fn=<DivBackward0>)
validation loss: 3.6228034019470217
validation accuracy: 8.585279399499584%
loss at step 600: tensor(3.3730, device='cuda:0', grad_fn=<DivBackward0>)
loss at step 800: tensor(3.3518, device='cuda:0', grad_fn=<DivBackward0>)
validation loss: 3.6040302085876466
validation accuracy: 8.731234361968307%
loss at step 1000: tensor(3.3579, device='cuda:0', grad_fn=<DivBackward0>)
loss at step 1200: tensor(3.3420, device='cuda:0', grad_fn=<DivBackward0>)
validation loss: 3.5527006022135414
validation accuracy: 8.997080900750626%
loss at step 1400: tensor(3.3205, device='cuda:0', grad_fn=<DivBackward0>)
loss at step 1600: tensor(3.2984, device='cuda:0', grad_fn=<DivBackward0>)
validation loss: 3.5205342896779377
validation accuracy: 8.

## Generating
This model has the Fast Wavenet Generation Algorithm (https://arxiv.org/abs/1611.09482) implemented. This might run faster on the cpu. You can give some starting data (of at least the length of receptive field) or let the model generate from zero. In my experience, a temperature between 0.5 and 1.0 yields the best results, but this may depend on the data set.

In [1]:
start_data = data[250000][0] # use start data from the data set
start_data = torch.max(start_data, 0)[1] # convert one hot vectors to integers

def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")

model.cpu()
generated = model.generate_fast(num_samples=160000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)

NameError: name 'data' is not defined

In [ ]:
import IPython.display as ipd

ipd.Audio(generated, rate=16000)